In [ ]:
#Supposed to be run on Colab
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece langchain langchain_community sentence-transformers huggingface_hub langchain-huggingface fastapi uvicorn pyngrok nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/7

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from huggingface_hub import login
from google.colab import userdata
from fastapi import FastAPI, Request
from pydantic import BaseModel
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
import nest_asyncio
import uvicorn
import threading
from google.colab import output
from pyngrok import ngrok
import json

In [7]:
login(token=userdata.get("HF_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

def load_quantized_model(model_name: str):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

model = load_quantized_model(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

In [11]:
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=32000,
    truncation=True,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

<ipython-input-11-96ed229c02b8>:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [12]:
CODE_ANALYSIS_PROMPT = PromptTemplate(
    template="""<s>[INST]
You are an expert code analyzer and reviewer with extensive knowledge across multiple programming languages and paradigms. Your task is to perform a deep and thorough analysis of the provided code snippet. Focus on providing a comprehensive evaluation that covers various aspects of code quality, security, and best practices.

Code to analyze:
{code}

Conduct your analysis based on the following criteria:

1. Syntax and Correctness:
   - Identify any syntax errors or incorrect language usage.
   - Evaluate the code's logical correctness and potential runtime errors.

2. Security and Vulnerabilities:
   - Identify potential security vulnerabilities (e.g., SQL injection, XSS, CSRF).
   - Assess the handling of sensitive data and potential exposure risks.
   - Evaluate input validation and sanitization practices.

3. Readability and Maintainability:
   - Assess code formatting, indentation, and overall structure.
   - Evaluate variable and function naming conventions.
   - Analyze the use of comments and documentation.
   - Compare against standard coding practices and style guides for the specific language.

4. Modularity and Code Organization:
   - Evaluate the code's modular structure and separation of concerns.
   - Assess the use of functions/methods and their single responsibility.
   - Analyze the overall architecture and potential for reusability.

5. Performance and Efficiency:
   - Identify potential performance bottlenecks or inefficient algorithms.
   - Evaluate time and space complexity of key operations.
   - Assess resource usage and potential optimizations.

6. Error Handling and Robustness:
   - Evaluate the implementation of error handling and exception management.
   - Assess the code's ability to handle unexpected inputs or situations.

7. Testability:
   - Assess how easily the code can be unit tested.
   - Evaluate the presence of dependency injection or other practices that facilitate testing.

8. Scalability:
   - Evaluate how well the code would perform under increased load or with larger datasets.
   - Assess potential limitations in the current implementation for scaling.

9. Code Smells:
   - Identify any code smells (e.g., duplicate code, long methods, excessive complexity).
   - Suggest refactoring opportunities to improve code quality.

10. Best Practices and Design Patterns:
    - Evaluate the use of language-specific and general programming best practices.
    - Identify the application of design patterns and assess their appropriateness.

Provide your analysis in a structured JSON format with the following schema:

{{
  "overall_rating": <number between 0 and 10>,
  "syntax_correctness": {{
    "rating": <number between 0 and 10>,
    "issues": [<list of identified issues>],
    "suggestions": [<list of improvement suggestions>]
  }},
  "security_vulnerabilities": {{
    "rating": <number between 0 and 10>,
    "identified_vulnerabilities": [<list of potential vulnerabilities>],
    "mitigation_suggestions": [<list of security improvement suggestions>]
  }},
  "readability_maintainability": {{
    "rating": <number between 0 and 10>,
    "strengths": [<list of positive aspects>],
    "areas_for_improvement": [<list of improvement suggestions>]
  }},
  "modularity_organization": {{
    "rating": <number between 0 and 10>,
    "analysis": "<brief analysis of modularity>",
    "suggestions": [<list of organizational improvement suggestions>]
  }},
  "performance_efficiency": {{
    "rating": <number between 0 and 10>,
    "bottlenecks": [<list of identified bottlenecks or inefficiencies>],
    "optimization_suggestions": [<list of performance improvement suggestions>]
  }},
  "error_handling_robustness": {{
    "rating": <number between 0 and 10>,
    "analysis": "<brief analysis of error handling>",
    "improvement_areas": [<list of suggestions for better error handling>]
  }},
  "testability": {{
    "rating": <number between 0 and 10>,
    "analysis": "<brief analysis of testability>",
    "suggestions": [<list of suggestions to improve testability>]
  }},
  "scalability": {{
    "rating": <number between 0 and 10>,
    "potential_issues": [<list of scalability concerns>],
    "improvement_suggestions": [<list of suggestions to improve scalability>]
  }},
  "code_smells": {{
    "identified_smells": [<list of identified code smells>],
    "refactoring_suggestions": [<list of refactoring recommendations>]
  }},
  "best_practices_patterns": {{
    "rating": <number between 0 and 10>,
    "positive_observations": [<list of well-applied practices or patterns>],
    "improvement_suggestions": [<list of suggestions for better adherence to best practices>]
  }},
  "summary": "<brief overall summary of the code quality and main points of improvement>"
}}

Ensure your analysis is thorough, specific, and actionable. Provide concrete examples from the code where relevant, and offer clear suggestions for improvement.

[/INST]</s>
""",
    input_variables=["code"]
)

code_analysis_chain = CODE_ANALYSIS_PROMPT | llm | StrOutputParser()

In [25]:
app = FastAPI()
nest_asyncio.apply()

@app.post('/analyze_code/')
async def conv(request: Request):
    data = await request.json()
    code = data.get("code")
    if not code:
        return {"error": "No code provided for analysis"}

    analysis_result = code_analysis_chain.invoke({"code": code})
    analysis_result = analysis_result.split("[/INST]</s>")
    analysis_result = analysis_result[-1]
    try:
        analysis_json = json.loads(analysis_result)
    except json.JSONDecodeError:
        return {"error": analysis_result}

    return analysis_json

In [26]:
from pyngrok import ngrok
NGROK_AUTH_TOKEN = "1flODqmi50WmfbgDuJ20LvQpC6z_5KBf3uiZdBxH84vauwTks"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
# !ngrok config add-authtoken 1flODqmi50WmfbgDuJ20LvQpC6z_5KBf3uiZdBxH84vauwTks

ngrok.kill()


In [27]:
import threading
from uvicorn import Config, Server
from pyngrok import ngrok
import time

NGROK_AUTH_TOKEN = "1flODqmi50WmfbgDuJ20LvQpC6z_5KBf3uiZdBxH84vauwTks"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()

time.sleep(4)

config = Config(app=app, host="127.0.0.1", port=8000, loop="asyncio")
server = Server(config)

def run_server():
  uvicorn.run(app, host="127.0.0.1", port=8000, loop="asyncio")

threading.Thread(target=run_server).start()

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

INFO:     Started server process [741]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Public URL: NgrokTunnel: "https://8778-34-32-213-77.ngrok-free.app" -> "http://localhost:8000"


In [24]:
import requests
import json

url = "https://f4a7-34-32-213-77.ngrok-free.app/"

code_to_analyze = """
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(10))
"""

response = requests.post(f"{url}/analyze_code/",
                         json={"code": code_to_analyze})

with open("output.json", "w") as file:
    json.dump(response.json(), file, indent=2)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 162, in startup
    server = await loop.create_server(
  File "/usr/lib/python3.10/asyncio/base_events.py", line 1519, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-23-cfc184609843>", line 16, in run_server
    uvicorn.run(ap

INFO:     34.32.213.77:0 - "POST /analyze_code/ HTTP/1.1" 200 OK
